## Gradient Vanishing / Exploding 
- Gradient Vanishing : 너무 작아지면서 그라디언트 소멸 (ReLU, sigmoid)
- Gradient Exploding : 그라디언트가 매우 커짐 -> 값이 너무나도 커질 경우 

### Solution
1. activation function을 바꾸기
2. careful initialization
- weight를 초기화 잘 하자. (ex xavier 초기화 등)
3. learning rate를 작게 하기 : Gradient Exploding 해결
4. batch normalization

## internal covariate shift 
- covariate shift : 뉴런 네트워크 학습 시 train과 test 셋의 분포가 실제로는 존재. -> 문제를 발생시킴
= 입력과 출력의 분포가 다르다. 
- 한 레이어 당 covariate shift 문제가 발생 
- 즉, 학습 도중 데이터의 분포가 층마다 계속 바뀌는 현상. => 이것 때문에 학습이 느려지고 불안정해짐. => batch normalization 사용

## Batch Normalization
: 뮤와 베리언스로 (평균과 분산으로) 정규화하는 법
- 각 레이어들 마다 정규화 레이어를 둬서 변형된 분포가 나오지 않도록 하기. 
- 각 mini batch 마다 normalization을 하겠다. = batch normalization
- 요약
1. 데이터가 들어오면 강제로 깔끔하게 정렬 (정규화) = 학습 안정화
2. 모델이 알아서 가장 학습하기 좋은 형태로 살짝 변형 (성능 유지)

## Train & eval mode
- train : dropout/batch normalization 기능 껐다 켰다 가능
- eval : dropout/batch normalization 껐다 켰다 기능 사용하지 않고 전체 데이터 사용

- model.eval() 안 쓸 경우?
1. Dropout: 예측할 때마다 결과값이 계속 바뀜. (랜덤성)
2. Batch Norm: 행동: 지금 들어온 데이터가 아니라, **"학습 때 미리 저장해 둔 전체 평균/분산(Running Stats)"**을 가져와서 정규화. 데이터 1개만 넣어서 예측하려 하면 에러가 나거나(분산 0), 완전히 엉터리 값이 나옴.


In [1]:
# Lab 10 MNIST and softmax
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pylab as plt

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(1)
if device == 'cuda':
    torch.cuda.manual_seed_all(1)

In [5]:
# parameters
learning_rate = 0.01
training_epochs = 10
batch_size = 32

In [6]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)
# dataset loader
train_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          drop_last=True)

In [10]:
linear1 = torch.nn.Linear(784, 32, bias=True)
linear2 = torch.nn.Linear(32, 32, bias=True)
linear3 = torch.nn.Linear(32, 10, bias=True)
relu = torch.nn.ReLU()
# 각 학습 파라미터가 다르기 때문에 다른 값 저장해서 다르게 선언 
bn1 = torch.nn.BatchNorm1d(32)
bn2 = torch.nn.BatchNorm1d(32)

nn_linear1 = torch.nn.Linear(784, 32, bias=True)
nn_linear2 = torch.nn.Linear(32, 32, bias=True)
nn_linear3 = torch.nn.Linear(32, 10, bias=True)

In [11]:
bn_model = torch.nn.Sequential(linear1, bn1, relu,
                               linear2, bn2, relu,
                               linear3).to(device)
nn_model = torch.nn.Sequential(nn_linear1, relu,
                                nn_linear2, relu,
                                nn_linear3).to(device)

In [13]:
criterion = torch.nn.CrossEntropyLoss().to(device)
bn_optimizer = torch.optim.Adam(bn_model.parameters(), lr=learning_rate)
nn_optimizer = torch.optim.Adam(nn_model.parameters(), lr=learning_rate)

In [ ]:
train_losses = []
train_accs = []

valid_losses = []
valid_accs = []

train_total_batch = len(train_loader)
test_total_batch = len(test_loader)
for epoch in range(training_epochs):
    bn_model.train()

    for X, Y in train_loader:
        X = X.view(-1, 784).to(device)
        Y = Y.to(device)
        
        bn_optimizer.zero_grad()
        bn_prediction = bn_model(X)
        bn_loss = criterion(bn_prediction, Y)
        bn_loss.backward()
        bn_optimizer.step()

        nn_optimizer.zero_grad()
        nn_prediction = nn_model(X)
        nn_loss = criterion(nn_prediction, Y)
        nn_loss.backward()
        nn_optimizer.step()
    
    with torch.no_grad():
        bn_model.eval()